In [1]:
from vif_agent.modules.identification.oracle import IdentificationOracleBoxModule
from openai import OpenAI
import os

from vif_agent.modules.search.search import VLMSearchModule

vlm_search_module = VLMSearchModule(
    client=OpenAI(
        api_key=os.environ.get("GOOGLE_API_KEY"),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    ),
    model="gemini-2.0-flash",
    temperature=0.3,
)

box_oracle_module = IdentificationOracleBoxModule(
    model="gemini-2.0-flash",
    temperature=0.3,
    client=OpenAI(
        api_key=os.environ.get("GOOGLE_API_KEY"),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    ),
)


def check_oracle(row):
    features = vlm_search_module.get_features(row["image_input"])
    current_oracle = box_oracle_module.get_oracle(
        features, row["instruction"], row["image_input"]
    )
    responses = [current_oracle(im_solution) for im_solution in row["image_solution"]]
    (
        row["valid"],
        row["edit_score"],
        row["added_condition"],
        row["remove_condition"],
        row["debug_ob"],
        row["full_report"],
    ) = ([], [], [], [], [], [])
    for res in responses:
        row["valid"].append(res[0])
        row["edit_score"].append(res[1][0])
        row["added_condition"].append(res[1][1])
        row["remove_condition"].append(res[1][2])
        row["full_report"].append(res[2])
        row["debug_ob"].append(res[3])
    return row

In [ ]:
from datasets import load_dataset

ds = load_dataset("CharlyR/varbench", "tikz", split="benchmark")

ds = ds.select_columns(["id","type","instruction","code","image_solution","image_input"])

ds = ds.filter(lambda row: row["type"]=="animal" ).train_test_split(test_size=0.1)["test"]

ds = ds.map(check_oracle)

ds.push_to_hub('CharlyR/OracleCheck')


/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using the latest cached version of the dataset since CharlyR/varbench couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'tikz' at /home/creux/.cache/huggingface/datasets/CharlyR___varbench/tikz/0.0.0/8cb86a70365f8fbffb37aa80db514f9f86b93be9 (last modified on Mon May 19 16:33:11 2025).
Parameter 'function'=<function check_oracle at 0x7985300074c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from

RateLimitError: Error code: 429 - [{'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.0-flash'}, 'quotaValue': '15'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '48s'}]}}]